# Инициализация

In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 24.5 MB/s 


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
drive.mount('/content/drive')
import re 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, RidgeClassifier, LogisticRegressionCV, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,StackingRegressor,StackingClassifier,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA,NMF
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel,VarianceThreshold,SelectKBest,chi2
from sklearn.pipeline import TransformerMixin
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR,SVC
from sklearn.metrics import mean_absolute_error,roc_auc_score
from sklearn.ensemble import StackingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputRegressor
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn.manifold import TSNE
from sklearn.naive_bayes import GaussianNB,CategoricalNB,MultinomialNB
from sklearn import set_config
from scipy.special import softmax

Mounted at /content/drive


# Загрузка

In [3]:
train=pd.read_csv("/content/drive/MyDrive/Kursk/train_dataset_train.csv")
test=pd.read_csv("/content/drive/MyDrive/Kursk/test_dataset_test.csv")

In [4]:
train['text']=list(map(lambda x: re.sub(r'\<[^>]*\>', ' ', x), train['Текст Сообщения'].tolist()))
train['text2']=train['text'].str.replace('&nbsp;',' ').str.replace('\n',' ').str.strip()
train['text5']=train['text'].str.replace('&nbsp;',' ').str.replace('\n',' ')

In [5]:
test['text']=list(map(lambda x: re.sub(r'\<[^>]*\>', ' ', x), test['Текст Сообщения'].tolist()))
test['text2']=test['text'].str.replace('&nbsp;',' ').str.replace('\n',' ').str.strip()
test['text5']=test['text'].str.replace('&nbsp;',' ').str.replace('\n',' ')

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [7]:
train['text5_l']=list(map(lemmatize, train['text5'].tolist()))
test['text5_l']=list(map(lemmatize, test['text5'].tolist()))

In [9]:
tdic=dict(zip(train['Категория'].astype('str').groupby(train['Тематика']).describe().index.tolist(),
train['Категория'].astype('str').groupby(train['Тематика']).describe().top.tolist()))

tdic['Неудовлетворительное материально-техническое обеспечение учреждения дополнительного образования']='7'
tdic['Необходимо строительство детской (спортивной) площадки в районе частного сектора']='3'
tdic['Дорожная разметка на дорогах регионального и межмуниципального значения']='0'
tdic['Несоответствие ценника цене товара']='13'
tdic['Несправедливое распределение мест в дошкольные учреждения']='7'
tdic['Отсутствие в населённом пункте сотовой связи']='9'
tdic['Плохое материально-техническое оснащение учреждений культуры и библиотек']='3'
tdic['Очистка  от снега и наледи  дорог в микрорайонах ИЖС']='0'
tdic['Меры поддержки в условиях неблагоприятной эпидемиологической ситуации в сфере занятости населения для частных лиц']='6'
tdic['Меры поддержки в условиях неблагоприятной эпидемиологической ситуации в сфере ЖКХ']='6'
tdic['Проблемы с обеспечением питанием в школах']='7'
tdic['Отсутствие твёрдого дорожного покрытия на дорогах регионального и межмуниципального значения']='0'

In [10]:
test['Категория t']=list(map(lambda x: int(tdic[x]),test['Тематика'].tolist()))

# CV

In [12]:
from sklearn.model_selection import KFold

In [14]:
test['Категория']=test['Категория t']

In [15]:
pipe=Pipeline([('input',ColumnTransformer([
               ('text1',CountVectorizer(ngram_range=(1,7),analyzer='char_wb',max_df=0.3,min_df=2),'text'),
               ('text2',CountVectorizer(ngram_range=(1,1),max_df=0.3,min_df=2),'text5_l'),
               ])),
               ('transformer',TfidfTransformer()),
               ('classifier',LinearSVC(C=1, class_weight='balanced'))])
test['Категория p']=0
cntt=0
for cnt in KFold(n_splits=50, random_state=1, shuffle=True).split(test):
  pipe.fit(pd.concat((train[train['Категория']!=12][['text','text5_l']],
                                    test.iloc[cnt[0]][['text','text5_l']])),
                    pd.concat((train[train['Категория']!=12]['Категория'],
                                    test['Категория'].iloc[cnt[0]])))
  
  test['Категория p'].iloc[cnt[1]]=pipe.predict(test.iloc[cnt[1]][['text','text5_l']])
  print('>>>',cntt)
  cntt+=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


>>> 0


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


>>> 1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


...

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


>>> 47


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


>>> 48
>>> 49


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
from sklearn.metrics import accuracy_score

In [16]:
accuracy_score(test['Категория t'], test['Категория p'])

0.851

# К отправке

In [17]:
set_config(display='diagram')

In [ ]:
pipe=Pipeline([('input',ColumnTransformer([
               ('text1',CountVectorizer(ngram_range=(1,7),analyzer='char_wb',max_df=0.3,min_df=2),'text'),
               ('text2',CountVectorizer(ngram_range=(1,1),max_df=0.3,min_df=2),'text5_l'),
               ])),
               ('transformer',TfidfTransformer()),
               ('classifier',LinearSVC(C=1, class_weight='balanced'))])
est=cross_val_score(pipe,pd.concat((train[train['Категория']!=12][['text','text5_l']],
                                    test[['text','text5_l']])),
                    pd.concat((train[train['Категория']!=12]['Категория'],
                                    test['Категория'])),cv=20)
print(est.mean())
print(est.std())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=20.
  UserWarning,


0.8359395973154362
0.025285085335848126


In [18]:
pipe.fit(pd.concat((train[train['Категория']!=12][['text','text5_l']],
                                    test[['text','text5_l']])),
                    pd.concat((train[train['Категория']!=12]['Категория'],
                                    test['Категория'])))

Pipeline(steps=[('input',
                 ColumnTransformer(transformers=[('text1',
                                                  CountVectorizer(analyzer='char_wb',
                                                                  max_df=0.3,
                                                                  min_df=2,
                                                                  ngram_range=(1,
                                                                               7)),
                                                  'text'),
                                                 ('text2',
                                                  CountVectorizer(max_df=0.3,
                                                                  min_df=2),
                                                  'text5_l')])),
                ('transformer', TfidfTransformer()),
                ('classifier', LinearSVC(C=1, class_weight='balanced'))])

In [19]:
from joblib import dump, load

In [20]:
dump(pipe, '/content/drive/MyDrive/Kursk/pipe.joblib') 

['/content/drive/MyDrive/Kursk/pipe.joblib']

In [ ]:
pipe=load('/content/drive/MyDrive/Kursk/pipe1.joblib') 

In [ ]:
test['Категория']=pipe.predict(test)

In [ ]:
test['Категория']=test['Категория p']

In [ ]:
answer=test[['id','Категория']]
answer.to_csv('sample_solution.csv',index=False)
answer

,id,Категория
0,843,3
1,1422,3
2,2782,3
3,2704,3
4,1,8
...,...,...
995,2619,16
996,2518,3
997,2524,0
998,1577,4
